# AMAT503:  Lecture 10

February 8, 2018.

Michael Lamoureux

# HEADS' UP!

I will be using [Jupyter notebooks](https://jupyter.org) in this classroom, to combine text, math, and graphics. 

Students can access Jupyter at [ucalgary.syzygy.ca](https://ucalgary.syzygy.ca)

An eBook on how to use syzygy is here: [intro.syzygy.ca](http://intro.syzygy.ca)

Lecture notes, code, assignments, etc are available on a git repo: [github.com/mlamoureux](https://github.com/mlamoureux)

The textbnook for the course is available electronically
[here](https://ucalgary.summon.serialssolutions.com/#!/#!%2Fsearch%3FbookMark=ePnHCXMwhZ3bCoJAEIYNvCjrGcq6CLoQFMLoNjN6gO4H6QBSWLRrh7fvH2ddC4QuxQ_Fw87-M7v7r-e4yFuPHXFTYoMxiNIYSbV43SxC_EMRdyEeV8mQFIlTMnQJOl9ogW5dIGGvc2RuPcdf52hBkJD-M-PNGLSvvzQdvs3AcfW9RDQdbdJdsg2MheubTAWEeBZXHLHXXiBEs6iByiIveOdjKvcR8YIWZChLhHZ2au87M-EzdUbIQTjSih6XahhT0c_zgR0Lq7ITslyyTPPMYKbC1FGfqkuZaZiUrhLId-gLgEMBZfCYbuI8QVFYjZPy9pmTVkC_dCvULKiyqL2VgbiIcrja03KI5h3PGfr3aj8nd5KS)

or [here](https://proquest-safaribooksonline-com.ezproxy.lib.ucalgary.ca/9780470183113?uicode=ucalgary).
Thanks to Phil for pointing this out. 


One of the links should take any student to the ucalgary login page and then to the web version of the book after they successfully login.

In [1]:
## Some startup commands

%matplotlib inline
from numpy import * 
from scipy import *
from matplotlib.pyplot import *
from IPython.display import IFrame
from IPython.display import Audio, display

## Summary

- Figure out the 2D wavelet transform, as matrix operations
- Relate that to matrix operations
- Refer to other notebook, Lecture09_code.ipynb for examples in connect
- Review connection between convolution coefficients, Fourier transforms and Z-transforms as polynomials (useful for assignment). 




## 1D wavelet transform, quick review

Recall that the 1D wavelet transform takes a vector of length N, multiplies by a special matrix (the wavelet matrix), and then outputs two vectors of size N/2 each. First one is the result of averaging or low-pass filtering (the "a" vector) and the other is the result of differencing, or high-pass filtering (the "d" vector). Schematically, we can write this in matrix-vector notation as
$$ W \left[\begin{array}{c} \vdots  \\ v \\  \vdots \end{array}  \right]
= \left[\begin{array}{c} a \\ \hline d \end{array} \right],$$
where $W$ is the $N\times N $ wavelet matrix, $v$ is a column vector, $a$ is the average vector and $d$ is the difference (detail) vector. The vectors $a$ and $d$ are each half the size of $v$, and so you can stack them up one on top of the other to get a column vector of size $N$.

Since $W$ is a matrix applied on the left, if we have a bunch of column vectors $v_1, v_2, v_3$ etc organized into an array, $W$ will act on each column separately. That is,
$$ W 
\left[\begin{array}{ccc} 
\vdots & \vdots & \vdots \\ 
v_1 & v_2 & v_3 \\  
\vdots & \vdots & \vdots \end{array}  \right]
= \left[\begin{array}{ccc} 
a_1 & a_2 & a_3 \\ 
\hline 
d_1 & d_2 & d_3 \end{array} \right],$$
where the $a_1$ is the averaged vector from column $v_1$, etc.


## 1D wavelet transform, on row vectors.

What is you want to act on row vectors? Well, just take the transpose of the last equation, so we get 
$$
\left[\begin{array}{ccc} 
\cdots & v_1 & \cdots \\ 
\cdots & v_2 & \cdots \\ 
\cdots & v_3 & \cdots
 \end{array}  \right] W^t
= \left[\begin{array}{c|c} 
a_1 & d_1  \\ 
a_2 & d_2  \\ 
a_3 & d_3  
\end{array} \right],$$
where here we think of the vectors $v,a,d$ as row vectors. 

Keep in mind, though, that the matrix that appears here is the transpose $W^t$ of the regular wavelet transform matrix.

## 2D wavelet transform

The idea of the 2D wavelet transform is to act on both rows and columns of an image array. Again we can do this schematically with the diagrams
$$W
\left[\begin{array}{ccc} 
 & \vdots &  \\ 
\cdots & v_{jk} & \cdots \\ 
 & \vdots & 
 \end{array}  \right] W^t
 = \left[\begin{array}{c|c} 
aa & ad  \\ 
\hline
da & dd \end{array} \right],$$
where the $v_{jk}$ are the entries of the original image of size $N\times N$ and each of the output matrices $aa, ad, da, dd$ are of size $N/2 \times N/2$ and are the result of averaging or differencing in either rows or columns. (4 choices, so four outputs).

Maybe for consistency of notation, I should write $aa_{jk}, ad_{jk} $ etc to match the $v_{jk}$ but you get the idea, I hope. 

## Data structures.
Whether you are in Matlab, or Py Wavelets toolkits, they will need some kind of data structure to give you the output in an organized way. Py Wavelets gives you a data structure that looks like this:
$$output = (aa, (da, ad, dd))$$
That is, the data structure has two elements, the first is the averaged matrix $aa$ and the second element is a 3-tuple with the other 3 matrices in it. They say they do this for ease in accessing the appropriate structures.

## Iterating the 2D wavelet transform.

As in the 1D case, we usually just iterate on the "averaged" output data. In the 2D case, this means iterating on the "aa" output matrix. So, in the schematic above
- start with initial image $v_{jk}$ with is an array of size $N\times N$
- apply $W$ and $W^t$ on left and right, to get the matrices $aa, ad, da, dd$
- to keep track, let's label them as $aa_1, ad_1, da_1, dd_1$
- To iterate, looks at $aa_1$ which is an array of size $N/2 \times N/2$. 
- apply the $N/2 \times N/2$ wavelet matrix to both sides of $aa_1$
- this gives you even smaller matrices $aa_2, ad_2, da_2, dd_2$
- now repeat, always applying the 2D transform to matrix $aa_k$ at the k-th level.

## Organizing the submatrices

It's kind of cute to reorganize these submatrices into one big matrix that shows how the various averages and details contain info about the original image. 

Trying to do this in LaTeX is a pain in the neck, but let's do it for fun. 

Note the sizes of the matrices reduce by a factor of 2 each time, so things squeeze in nicely.

$$
\left[
\begin{array}{c|c}
\begin{array}{c|c}
aa_2 & da_2 \\
\hline
ad_2 & dd_2
\end{array}
& da_1 \\
\hline
ad_1 & dd_1
\end{array}
\right].
$$
Pywavelet outputs this as a data structure with the various levels. Something like this:
$$(aa_2, (ad_2, da_2, dd_2), (ad_1, da_1, dd_1)).$$
Or maybe it is in this order:
$$(aa_2, (ad_1, da_1, dd_1), (ad_2, da_2, dd_2)).$$

Anyhow, the point is the data strcture has three element. The first is the $aa$ matrix, and the next two elements are 3-tuples of the various detail matrices. 

You really should read the docs, and should I. But really, I only read it when I am programming in code, since there is too much to memorize!




## Discrete convolution, filters, polynomials, Fourier transforms

We talked about this briefly in class. 

I want you to  notice that there are many ways to think about, and to analyse our convolution filters.

If you have a filter with coeffcients $(h_0, h_1 h_2, \cdots, h_n)$ you can write down its Fourier transform as $$H(\omega) = \sum_k h_k e^{2\pi i \omega}$$ or its Z-transform
$$H(Z) = \sum_k h_k Z^k$$ which we notice is a polynomial in (complex) variable $Z$.

The Fourier transform is just that polynomial evaluated on points on the unit circle, given by the substitution $$Z = e^{2\pi i \omega}.$$ So everything you want to know about the Fourier transform of the filter can be seen from this polynomial. 

Polynomials, we know can be factored as a product of linear terms $(z-z_k)$, where the points $z_k$ are called the roots of the polynomials. (Or, you can call them  the zeros of the polynomial.) So if you know the zeros of the polynomial, you know pretty much everything there is to know about the polynomial. 

Don't forget that if the filter coefficients $h_k$ are all real, then the roots of the polynomial have to come in complex conjugate pairs.

Anyhow, this is a pretty powerful idea. If we want to design a convolution filter, all we have to do is think about zeros of a polynomial.

For instance, when Daubechies was designing her high pass filter, she knew she had to put a zero at freqency $\omega = 0$. This means the polynomial had to have a zero (or roots) at $ z= e^{2\pi i 0}$ which means a root at $z =1$. The lowpass filter has to have a zero at Nyquist frequency, so we need a root at $z = e^{2\pi i 0.5}$, which is a root at $z=-1$.

We notice the Haar highpass filter has coefficients ($1/2, -1/2)$ which corresponds to the polynomial $(1-z)/2$ which has a root at 1 as expected. The Haar lowpass filter has coefficients $(1/2, 1/2)$ corresponding to polynomial $(1+z)/2$ which has a root at -1. 

For higher order Daubechies filters, recall we ask for the Fourier transform to have a zero and and zero-derivative at the relevant frequencies. This means the polynomial has to have a double root. For instance, the high pass filter will have a factor like $(z-1)^2$ appearing. 



## Choice of roots.

It may seem surprising, but different polynomials can have the same filter response -- at least in terms of amplitude. 

Well, maybe not too surprising. A filter with three coefficients $(0, 2, 1)$ has the same amplitude response as $(2,1)$ since the first has FT 
$$H_1(\omega) = 2e^{2\pi i \omega} + e^{4\pi i \omega} = (2+ e^{2\pi i \omega})e^{2\pi i \omega}$$
while the second has FT 
$$H_2(\omega) = 2+ e^{2\pi i \omega}.$$
These only differ by a factor of $e^{2\pi i \omega}$, so when you take absolute values, that factor disappears. The filters do introduce a different delay in the output, but that doesn't show up in the amplitude response. 

More interestingly, the two filters with coefficients $(2,1)$ and $(1,2)$ also also have the same amplitude response. You should check this yourself: One has FT
$$H_1(\omega) = 2+ e^{2\pi i \omega}$$ while the other has
$$H_2(\omega) = 1+ 2e^{2\pi i \omega}.$$ Can you do some algebra with complex numbers to convince yourself that 
$$| H_1(\omega) | = | H_2(\omega) |$$
for all $\omega$?

These two filters have zeros at different places. The first at $z=-2$, the second at $z = -1/2$. Notice these two zeros are reciprocals of each others.

This is a general results. In the case of real zeros, flipping a zero with its reciprocal will not change the amplitude response of a (finite) convolutional filter. Its slightly more complicated for complex roots, but when they come in conjugate pairs, it works out nicely. (You have to move both zeros in the pair.)

For various reasons, we often choose to take the roots outside the open unit disk in the complex plane. Daubechies hopefully did this -- the assignment for next week asks you to compute the zeros for her filters. We would like to know where they are in the complex plane.

One of these reasons is that engineers like minimum phase delay filters. Such filters cause most of the energy in a signal to pass as quickly as possible. That might be a good thing if you are filtering music or speech -- you don't want it to be delayed for unnecessary processing. 